In [1]:
# Dependencies
import requests
import json
import pandas as pd

In [2]:
#check url response
apikey ="f98921da60b71947056633acb25f5147"
search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id=259&entity_type=city"
print(requests.get(search_url, headers={"user-key" : apikey}))



<Response [200]>


In [3]:
#check data structure
# Retrieving data and converting it into JSON
data = requests.get(search_url, headers={"user-key" : apikey}).json()
#display data structure
data


{'results_found': 17498,
 'results_start': 0,
 'results_shown': 20,
 'restaurants': [{'restaurant': {'R': {'has_menu_status': {'delivery': -1,
      'takeaway': -1},
     'res_id': 16577492,
     'is_grocery_store': False},
    'apikey': 'f98921da60b71947056633acb25f5147',
    'id': '16577492',
    'name': 'Chin Chin',
    'url': 'https://www.zomato.com/melbourne/chin-chin-3-cbd?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
    'location': {'address': '125 Flinders Lane, CBD, Melbourne',
     'locality': 'CBD',
     'city': 'Melbourne',
     'city_id': 259,
     'latitude': '-37.8155645773',
     'longitude': '144.9703757465',
     'zipcode': '3000',
     'country_id': 14,
     'locality_verbose': 'CBD, Melbourne'},
    'switch_to_order_menu': 0,
    'cuisines': 'Thai, Asian Fusion',
    'timings': '11 AM to 11 PM (Mon-Thu, Sun), 11 AM to 11:30 PM (Fri-Sat)',
    'average_cost_for_two': 135,
    'price_range': 4,
    'currency': '$',
    'highlights': ['Dinner',
     'Cr

In [4]:
resto_num = data["results_found"]
print(f"Number of Rest: {resto_num}")

Number of Rest: 17498


In [5]:
#single city 
#print(json.dumps(data, indent=4, sort_keys=True))

rest_id_list = []
rest_name_list =[]
rest_cuis_list = []
rest_local_list = []
rest_city_list = []
rest_rate_list = []
rest_delivery_list = []
rest_ratenum_list = []

resto_num = 20
show_page = 0

citynum =259

search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
data = requests.get(search_url, headers={"user-key" : apikey}).json()
for page in range(5):
    for count in range(resto_num):
        try: 
            rest_id = data["restaurants"][count]["restaurant"]["id"]
            rest_name = data["restaurants"][count]["restaurant"]["name"]
            rest_cuis = data["restaurants"][count]["restaurant"]["cuisines"]
            rest_local = data["restaurants"][count]["restaurant"]["location"]["locality"]
            rest_city = data["restaurants"][count]["restaurant"]["location"]["city"]
            rest_rate_num = data["restaurants"][count]["restaurant"]["all_reviews_count"]
            rest_rate = data["restaurants"][count]["restaurant"]["user_rating"]["aggregate_rating"]
            rest_delivery = data["restaurants"][count]["restaurant"]["highlights"]
            
            #check if current restaurant does delivery 
            theydelivery = False
            for doesdelivery in rest_delivery:
                if doesdelivery == 'Delivery':
                    theydelivery = True
            
            if theydelivery == True:
                rest_delivery = "Yes"
            else:
                rest_delivery = "No"
            
            #append to list
            rest_id_list.append(rest_id)
            rest_name_list.append(rest_name)
            rest_cuis_list.append(rest_cuis)
            rest_local_list.append(rest_local)
            rest_city_list.append(rest_city)
            rest_rate_list.append(rest_rate)
            rest_ratenum_list.append(rest_rate_num)
            rest_delivery_list.append(rest_delivery)
            
            #go to next page
            if count == 19:
                show_page = show_page + 20
                search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
                data = requests.get(search_url, headers={"user-key" : apikey}).json()
                count = 0
        except:
            print("error")
#check errors when making dataframe
try:            
    restaurants_df = pd.DataFrame({
        "Restaurant ID":rest_id_list,
        "Name":rest_name_list,
        "Cuisine":rest_cuis_list,
        "Locality":rest_local_list,
        "City":rest_city_list,
        "Rating":rest_rate_list,
        "Total Number Ratings": rest_ratenum_list,
        "Delivery":rest_delivery_list
    })
except:
    print(len(rest_id_list))
    print(len(rest_name_list))
    print(len(rest_cuis_list))
    print(len(rest_local_list))
    print(len(rest_city_list))
    print(len(rest_rate_list))
    print(len(rest_review_list))
    print(len(rest_delivery_list))
    

restaurants_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery
0,16577492,Chin Chin,"Thai, Asian Fusion",CBD,Melbourne,4.6,3205,No
1,16573665,The Hardware Societe,"French, Cafe Food, Coffee and Tea",CBD,Melbourne,4.6,2104,No
2,16585905,Tipo 00,Italian,CBD,Melbourne,4.9,721,No
3,16583098,Lune Croissanterie,"Bakery, French",Fitzroy,Melbourne,4.9,753,No
4,16582897,Supernormal,"Asian, Fusion",CBD,Melbourne,4.5,1022,No
...,...,...,...,...,...,...,...,...
95,16571710,Laksa King,"Malaysian, Chinese",Flemington,Melbourne,4.6,1303,No
96,16586156,The Grand Trailer Park Taverna,"Burger, American",CBD,Melbourne,4.4,668,Yes
97,16579874,Two Birds One Stone,"Coffee and Tea, Modern Australian, Cafe Food",South Yarra,Melbourne,4.4,684,No
98,18716100,Mansae Korean BBQ & Bar,"Korean, Korean BBQ",CBD,Melbourne,4.5,225,No


In [6]:
#save dataframe single city to csv
restaurants_df = restaurants_df.drop_duplicates()
restaurants_df.to_csv("single_restaurant_data.csv", encoding="utf-8", index=False)

In [7]:
#test looper
rest_delivery = data["restaurants"][count]["restaurant"]["highlights"]
rest_delivery
delivery = False
for check in rest_delivery:
    if check == 'Breakfast':
        delivery = True
if delivery == True:
    print ("yes")

IndexError: list index out of range

In [9]:
#multiple cities

#print(json.dumps(data, indent=4, sort_keys=True))
cityid = [296,259,260]

rest_id_list = []
rest_name_list =[]
rest_cuis_list = []
rest_local_list = []
rest_city_list = []
rest_rate_list = []
rest_delivery_list = []
rest_ratenum_list = []

resto_num = 20
loop_num = 0
show_page = 0

for citynum in cityid:
    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
    data = requests.get(search_url, headers={"user-key" : apikey}).json()
    for page in range(5):
        for count in range(resto_num):
            try: 
                rest_id = data["restaurants"][count]["restaurant"]["id"]
                rest_name = data["restaurants"][count]["restaurant"]["name"]
                rest_cuis = data["restaurants"][count]["restaurant"]["cuisines"]
                rest_local = data["restaurants"][count]["restaurant"]["location"]["locality"]
                rest_city = data["restaurants"][count]["restaurant"]["location"]["city"]
                rest_rate_num = data["restaurants"][count]["restaurant"]["all_reviews_count"]
                rest_rate = data["restaurants"][count]["restaurant"]["user_rating"]["aggregate_rating"]
                rest_delivery = data["restaurants"][count]["restaurant"]["highlights"]
                
                #check if current restaurant does delivery 
                theydelivery = False
                for doesdelivery in rest_delivery:
                    if doesdelivery == 'Delivery':
                        theydelivery = True

                if theydelivery == True:
                    rest_delivery = "Yes"
                else:
                    rest_delivery = "No"

                #append to list
                rest_id_list.append(rest_id)
                rest_name_list.append(rest_name)
                rest_cuis_list.append(rest_cuis)
                rest_local_list.append(rest_local)
                rest_city_list.append(rest_city)
                rest_rate_list.append(rest_rate)
                rest_ratenum_list.append(rest_rate_num)
                rest_delivery_list.append(rest_delivery)
                
                #go to next page
                if count == 19:
                    show_page = show_page + 20
                    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
                    data = requests.get(search_url, headers={"user-key" : apikey}).json()
                    count = 0
            except:
                print("error")
    count = 0
    page = 0
    show_page = 0


#check errors when making dataframe
try:            
    restaurants_df = pd.DataFrame({
        "Restaurant ID":rest_id_list,
        "Name":rest_name_list,
        "Cuisine":rest_cuis_list,
        "Locality":rest_local_list,
        "City":rest_city_list,
        "Rating":rest_rate_list,
        "Total Number Ratings": rest_ratenum_list,
        "Delivery":rest_delivery_list
    })
except:
    print(len(rest_id_list))
    print(len(rest_name_list))
    print(len(rest_cuis_list))
    print(len(rest_local_list))
    print(len(rest_city_list))
    print(len(rest_rate_list))
    print(len(rest_review_list))
    print(len(rest_delivery_list))
restaurants_df = restaurants_df.drop_duplicates()
restaurants_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery
0,16600038,Bib & Tucker,"Modern Australian, Cafe Food",North Fremantle,Perth,4.1,692,No
1,16597645,Mrs. S,"Patisserie, Cafe Food",Maylands,Perth,4.5,643,No
2,18196379,tbsp.,"Sandwich, Coffee and Tea, Cafe Food",Bayswater,Perth,4.7,335,No
3,16598431,Tuck Shop Cafe,"Modern Australian, Cafe Food",Northbridge,Perth,4.8,799,No
4,16599082,Sayers Sister,"Modern Australian, Coffee and Tea",Perth CBD,Perth,4.6,496,No
...,...,...,...,...,...,...,...,...
295,18494965,Sasaki,Japanese,CBD,Sydney,4.3,67,No
296,16558672,Pancakes On The Rocks,"Grill, Desserts, Pizza, Cafe Food, Coffee and Tea",The Rocks,Sydney,4.1,324,No
297,16559582,Spice Temple,"Chinese, Fusion, Yum Cha",CBD,Sydney,4.3,243,No
298,16563714,Orto Trading Co.,"Modern Australian, Cafe Food, Coffee and Tea",Surry Hills,Sydney,4.3,212,No


In [10]:
#save dataframe multi city to csv 
restaurants_df = restaurants_df.drop_duplicates()
restaurants_df.to_csv("multi_restaurant_data.csv", encoding="utf-8", index=False)

In [41]:
restaurants_df = restaurants_df.sort_values(by=["Rating"],ascending=False)
restaurants_df

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery
249,16562433,Cow & The Moon,"Ice Cream, Desserts, Coffee and Tea",Enmore,Sydney,4.9,355,No
278,15545439,Manpuku - Chatswood,"Japanese, Ramen",Chatswood,Sydney,4.9,196,No
265,16566535,Lorraine's Patisserie,"Desserts, Bakery","Ivy, CBD",Sydney,4.9,111,No
8,16597513,Pacific Rim Mix Plate,"Hawaiian, Japanese",Applecross,Perth,4.9,213,No
291,16568476,La Mamma del Gelato Anita,Ice Cream,"Central Park, Chippendale",Sydney,4.9,231,No
...,...,...,...,...,...,...,...,...
16,16599949,Gordon St Garage,"Modern Australian, Cafe Food, Coffee and Tea",West Perth,Perth,4.0,787,No
98,16601688,Tropico,"Tapas, Modern Australian, Pizza",North Beach,Perth,4.0,302,No
48,16600326,Mary Street Bakery,Bakery,Highgate,Perth,4.0,518,No
76,16599146,The Merrywell,"American, Burger, Bar Food",Burswood,Perth,4.0,619,No


In [29]:
#get top restaurant per city
perth_top = restaurants_df.loc[restaurants_df["City"]=="Perth"]
perth_top = perth_top.head()
perth_top

,Restaurant ID,Name,Cuisine,Locality,City,Rating,Total Number Ratings,Delivery
51,16598837,Run Amuk,Fast Food,"Orient Street, South Fremantle",Perth,4.9,341,No
8,16597513,Pacific Rim Mix Plate,"Hawaiian, Japanese",Applecross,Perth,4.9,213,No
91,16598976,Marumo,"Japanese, Seafood, Modern Australian",Nedlands,Perth,4.9,141,No
3,16598431,Tuck Shop Cafe,"Modern Australian, Cafe Food",Northbridge,Perth,4.8,799,No
28,16598168,Nobu Perth,"Japanese, Sushi",Burswood,Perth,4.8,663,No
